In [2]:
import requests
import pandas as pd
import numpy as np
import sddk
import spacy
import zipfile
import io
import re

In [3]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [4]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_cleaned_dated_2020-12-11.json", "df", conf)

In [5]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,...,clean_text_interpretive_sentence,clean_text_pythia,raw_date,dating_dict,not_before,not_after,or,date_tags,certainty,link
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,{},12,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκε...,c. 510-500 a.,"{'or': None, 'not_before': -511, 'not_after': ...",-511.0,-499.0,None,"[range, phase, ca]",None,None
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,{},14,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,...,"․․ ․․․ν βολ ․ ․․ α ⋮ ℎεκον σιον, γνοσθε͂ι δὲ ν...",ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄλ...,c. 500 a.,"{'or': None, 'not_before': -505, 'not_after': ...",-505.0,-495.0,None,"[exact, phase, ca]",None,None
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,{},13,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,...,αρι ℎερακλειο․․․․ αρο τιθέναι τὸς ἀθλοθέτας τρ...,αρι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκον...,490-480 a.,"{'or': None, 'not_before': -490, 'not_after': ...",-490.0,-480.0,None,[range],None,None
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,{},56,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,...,․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․ ․․․․․․․․․...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,485/4 a.,"{'or': None, 'not_before': -485, 'not_after': ...",-485.0,-484.0,None,[range],None,None
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,{},6,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,...,ἔδοχσεν ⋮ τε͂ι βολε͂ι ⋮ καὶ το͂ι δέμοι ⋮ ℎότε ...,ἔδοχσεν τει βολει καὶ τοι δέμοι ὅτε παραιβάτες...,c. 500 a.,"{'or': None, 'not_before': -505, 'not_after': ...",-505.0,-495.0,None,"[exact, phase, ca]",None,None


In [6]:
len(PHI[PHI["clean_text_pythia"] ==  PHI["clean_text_interpretive_word"]])

1144

In [7]:
# when pythia not available, use our text

def extend_pythia(row):
    pythia = row["clean_text_pythia"]
    if pythia == "":
        pythia = row["clean_text_interpretive_word"]
    return pythia

PHI["clean_text_pythia_ext"] = PHI.apply(lambda row: extend_pythia(row), axis=1)

In [8]:
### to do below some matching, we have to take care of appropriate encoding
from unicodedata import normalize
def normalize_string(string):
    try:
        return normalize("NFC", string)
    except: 
        return ""
PHI["string_normalized"] = PHI["clean_text_pythia_ext"].apply(normalize_string)

In [9]:
# sentences
# add column with sentences
v_ud = "’"
v_agt = "ʼ"

def get_sentences(string):
    splits = re.split(r"(\.|\·|\:|\;|\?|\!)" , string)
    sentences = []
    for n in range(0, len(splits), 2):
        try:
            sentence = splits[n]+splits[n+1]
            sentence = "".join([c for c in sentence if re.match("[\w|\s|\.|\,|\·|\:|\;|\?|\!]", c)]) 
            sentence = re.sub("^\s*", "", sentence)
            sentence = re.sub(v_ud, v_agt, sentence)
            sentences.append(sentence)
        except:
            sentence = splits[n]
            sentence = re.sub("^\s*", "", sentence)
            if len(sentence) > 0:
                sentences.append(sentence)
            break
    return sentences

In [10]:
PHI["sents"] = PHI["string_normalized"].apply(get_sentences)

In [11]:
PHI["sents_N"] = PHI["sents"].apply(lambda x: len(x))

In [12]:
len(PHI[PHI["sents_N"] > 1])

22321

In [13]:
len(PHI[PHI["sents_N"] > 2])

11125

In [14]:
# look at some inscriptions having more than two sentences.
PHI[PHI["sents_N"] > 2]["sents"].tolist()[:5]

[['δραχμεισι τες τὸς ιο μενος δεμο τον πόλεον δοκει ἀνατιθα ἐάν τις τον ον ἒ ὁσαλ ο ἒ ενορ νας ἵνα αντοιν σ τον πόλεον ταυτ χρε μεμβολ εχθεοχε ν δενλο ἐὰν δὲ μέ ἑκαστοσε αιν ιμιας πε ετατεν ιαο λπο σειανλλαμ ολει καὶ μὲ νεοτερ χρέσθο τοι ἱεροι ἐὰν δὲ ι μὲ χρέσθο ἐὰν δὲ ἰ κατὰ ταὐτὰ ταῦτα ἐὰν πλεῖστον κατὰ τὲν δύναμιν πρᾶχσαι δ ἔκπραχσιν ἐὰν δὲ μὲ ἐγδοι τὸν ὀφλόντα μὲ χρέσθο τοι ἱεροι ἐὰν ἀμφισβετοσι μὲ κλεθεναι ἐμ πόλει εν ἐλθοσαν ἀδικ ὕστερον ἑ βολὲ α ι τὸν ἀθεναῖον μὲ ἐκ γες πο τούτον τον πόλεον μεδὲ ἁμος βιᾶσθαι ἐὰν μὲ δίκεν ὀφλόντα ἐπιχορίαν ἒ ἐς πολεμίος λεφθέντα ἕτις δ ἂν τομ πόλεον μὲ ἐθέλει δίκας διδόναι καὶ δέχεσθαι ἀθεναίοισιν ἀπὸ χσυμβολον.',
  'ια ντο τος τὰ μὲν ἁκόσια ἁπλει τὰ δὲ ἡ εκόσια διπλει σπονδὰς εἶναι τοῖσι μύστεσιν καὶ τοῖς ἐπόπτεισιν καὶ τοῖς ἀκολούθοισιν καὶ χρέμασιν τον ὀθνείον καὶ ἀθεναῖοισιν ἅπασιν ἄρχεν δὲ τὸν χρόνον τον σπονδον το μεταγειτνιονος μενὸς ἀπὸ διχομενίας καὶ τὸν βοεδρομιονα καὶ το υανοφσιονος μέχρι δεκάτες ἱσταμένο τὰς δὲ σπονδὰς εἶναι ἐν τεισι 

# Download spacy model from sciencedata, load it etc.

In [15]:
# download our spacy model and unzip it.
#!curl https://sciencedata.dk//shared/b8f6c317c433385ce79a9531a0fbf658?download -o ../data/spacy_grc_model.zip
#!unzip  -: ../data/spacy_grc_model.zip

In [16]:
nlp = spacy.load("../models/spacy_grc_model_7")
path = "../models/spacy_grc_model_7/grc_lemmatization.py"
exec(open(path).read())
nlp.add_pipe(grc_doc_lemmatizer, "grc_doc_lemmatizer", after="tagger")

In [17]:
def apply_nlp(sentences_list):
    spacy_docs = []
    for sentence in sentences_list:
        doc = nlp(sentence)
        spacy_docs.append(doc)
    return spacy_docs

In [18]:
PHI["sents"].tolist()[:3]

[['ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκεν ἐᾶ σαλαμῖνι λεν ἀθένεσι τελεν καὶ στρατεύεσθαι τὰ δ ἐ σαλαμῖνι μὲ μισθον ἐὰ μὲ οἰκο μισθόμενο ἐὰν δὲ μισθοι ἀποτίνεν τὸ μισθόμενον καὶ τὸ μισθοντα ἑκάτερον ἐς δεμόσιον ἐσπράτεν δὲ τὸν ἄρχοντα ἐὰν δὲ μέ εὐθύνεσθαι τὰ δὲ ὅπλα παρέχεσθαι αὐτὸς τριάκοντα δραχμον ὁπλισμένον δὲ τὸν ἄρχοντα τὰ ὅπλα κρίνεν ἐπὶ τες βολες .'],
 ['ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄλλοθι δὲ μεδὲ ἁμο κελεύοσι χινχάνεν ἐπιδικάζεν κατ ἐπιδικαζέτο τὲν λο ὅταν ἁρκτοῦρος ἐπιτέλλεται ν τρέπεν ἴστο ἱ οιας τἀσυρε καὶ τέταρας ὀβελός ὑ ὑπὲρ πεντέκοντα κ .'],
 ['αρι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκοντα ἄνδρας δὲ τὸν ἀγονα ἐπιόφσασθαι ἐκ τον ἐπιδέμομ τρες ἐκ φυλες ἑκάστες ὑποσχομένος ἐν τοι ἱεροι ὁς ἂν οἷόν τ ει χσυνδιαθέσεν τὸν ἀγονα μὲ ὄλεζον ἒ τριάκοντα ἔτε γεγονότας τούτος δὲ τὸς ἄνδρας ὀμόσαι ἐν τοι ἱεροι καθ ἱερον ἐπιστατεσαι δὲ.']]

In [20]:
nlp(PHI["sents"].tolist()[0][0])

ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκεν ἐᾶ σαλαμῖνι λεν ἀθένεσι τελεν καὶ στρατεύεσθαι τὰ δ ἐ σαλαμῖνι μὲ μισθον ἐὰ μὲ οἰκο μισθόμενο ἐὰν δὲ μισθοι ἀποτίνεν τὸ μισθόμενον καὶ τὸ μισθοντα ἑκάτερον ἐς δεμόσιον ἐσπράτεν δὲ τὸν ἄρχοντα ἐὰν δὲ μέ εὐθύνεσθαι τὰ δὲ ὅπλα παρέχεσθαι αὐτὸς τριάκοντα δραχμον ὁπλισμένον δὲ τὸν ἄρχοντα τὰ ὅπλα κρίνεν ἐπὶ τες βολες .

In [22]:
PHI.iloc[:100]["sents"].apply(apply_nlp)

0     [(ἔδοχσεν, τοι, δέμοι, τὸς, ἐ, σαλαμῖνι, κλερό...
1     [(ν, βολ, α, ἑκον, σιον, γνοσθει, δὲ, ν, ἀτεχν...
2     [(αρι, ἑρακλειο, αρο, τιθέναι, τὸς, ἀθλοθέτας,...
3     [(ἐὰν, δέ, τις, αν, ἒ, φρορὰν, μὲ, πεντέκοντα,...
4     [(ἔδοχσεν, τει, βολει, καὶ, τοι, δέμοι, ὅτε, π...
                            ...                        
95    [(καλλίππο, θετταλο, .), (ἔδοξεν, τῆι, βολῆι, ...
96    [(ναν, νν, ο, ο, διαχεροτονεσαι, τὸν, δεμον, α...
97    [(θεοί, ἔδοχσεν, τε͂ι, βολε͂ι, καὶ, το͂ι, δέμο...
98    [(ἔδοξεν, τῆι, βολῆι, καὶ, τῶι, δήμωι, ἀντιοχὶ...
99    [(θεοί, ἔδοχσεν, τει, βολει, καὶ, τοι, δέμοι, ...
Name: sents, Length: 100, dtype: object

In [24]:
%%time
def try_apply_nlp(sents):
    try: return apply_nlp(sents)
    except: return None

## apply it on the normalized string
PHI_docs_series = PHI["sents"].apply(try_apply_nlp)

CPU times: user 20min 53s, sys: 3.94 s, total: 20min 57s
Wall time: 20min 58s


In [32]:
for token in PHI_docs_series.iloc[0][0]:
    print(token.text, token.pos_, token.lemma_)

ἔδοχσεν VERB ἔδοχσεν
τοι PRON σύ
δέμοι ADJ δέμοι
τὸς PRON τὸς
ἐ INTJ ἒ
σαλαμῖνι NOUN Σαλαμίς
κλερόχος NOUN κλερόχος
οἰκεν VERB οἰκεν
ἐᾶ INTJ ἔα
σαλαμῖνι NOUN Σαλαμίς
λεν VERB λεν
ἀθένεσι NOUN ἀθένεσι
τελεν VERB τελεν
καὶ CCONJ καί
στρατεύεσθαι VERB στρατεύω
τὰ PRON ὁ
δ PRON δ
ἐ INTJ ἒ
σαλαμῖνι NOUN Σαλαμίς
μὲ PRON ἐγώ
μισθον NOUN μισθός
ἐὰ INTJ ἔα
μὲ PRON ἐγώ
οἰκο VERB οἰκο
μισθόμενο VERB μισθόμενο
ἐὰν SCONJ ἐάν
δὲ ADV δέ
μισθοι NOUN μισθός
ἀποτίνεν VERB ἀποτίνεν
τὸ DET ὁ
μισθόμενον VERB μισθόμενον
καὶ CCONJ καί
τὸ DET ὁ
μισθοντα VERB μισθοντα
ἑκάτερον ADJ ἑκάτερος
ἐς ADP εἰς
δεμόσιον ADJ δεμόσιον
ἐσπράτεν VERB ἐσπράτεν
δὲ ADV δέ
τὸν DET ὁ
ἄρχοντα NOUN ἄρχων
ἐὰν SCONJ ἐάν
δὲ ADV δέ
μέ PRON ἐγώ
εὐθύνεσθαι VERB εὐθύνω
τὰ DET ὁ
δὲ ADV δέ
ὅπλα NOUN ὅπλον
παρέχεσθαι VERB παρέχω
αὐτὸς ADJ αὐτός
τριάκοντα NUM τριάκοντα
δραχμον ADJ δραχμον
ὁπλισμένον VERB ὁπλισμένον
δὲ ADV δέ
τὸν DET ὁ
ἄρχοντα NOUN ἄρχων
τὰ DET ὁ
ὅπλα NOUN ὅπλον
κρίνεν VERB κρίνεν
ἐπὶ ADP ἐπί
τες NOUN τες
βολες NOUN βολες
. PU

In [37]:
def extraxt_lem_sents(sentences):
    lemmatized_sentences = []
    try:
        for sent in sentences:
            lemmata = []
            for token in sent:
                if token.pos_ in ["NOUN", "PROPN", "ADJ", "VERB"]:
                    lemmata.append(token.lemma_)
            lemmatized_sentences.append(lemmata)
    except: pass
    return lemmatized_sentences

In [38]:
%%time
lem_sents_series = PHI_docs_series.apply(extraxt_lem_sents)

CPU times: user 8.61 s, sys: 212 ms, total: 8.83 s
Wall time: 8.83 s


In [39]:
%%time
PHI["lem_sents"] = lem_sents_series
#PHI["lemmata"] = PHI["string_normalized"].apply(lambda x: lemmatize_string(x, all_lemmata=False, filter_by_postag=["n", "a", "v"], involve_unknown=True))

CPU times: user 5.13 ms, sys: 0 ns, total: 5.13 ms
Wall time: 4.86 ms


In [40]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
PHI["lemmata"] = PHI["lem_sents"].apply(flat_lemmata)

In [41]:
sddk.write_file("SDAM_data/PHI/PHI_dated_lemmatized_20201214.json", PHI, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_dated_lemmatized_20201214.json"


In [ ]:
PHI.columns

In [44]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,...,or,date_tags,certainty,link,clean_text_pythia_ext,string_normalized,sents,sents_N,lem_sents,lemmata
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,{},12,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,...,None,"[range, phase, ca]",None,None,ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκε...,ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκε...,[ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκ...,1,"[[ἔδοχσεν, δέμοι, Σαλαμίς, κλερόχος, οἰκεν, Σα...","[ἔδοχσεν, δέμοι, Σαλαμίς, κλερόχος, οἰκεν, Σαλ..."
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,{},14,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,...,None,"[exact, phase, ca]",None,None,ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄλ...,ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄλ...,[ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄ...,1,"[[βολ, ἑκών, σίον, γνοσθει, ἄτεχνος, μεδὲ, κελ...","[βολ, ἑκών, σίον, γνοσθει, ἄτεχνος, μεδὲ, κελε..."
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,{},13,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,...,None,[range],None,None,αρι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκον...,αρι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκον...,[αρι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,1,"[[ἀρίς, ἑρακλειο, ἀρόω, τίθημι, ἀθλοθέτης, ἀνή...","[ἀρίς, ἑρακλειο, ἀρόω, τίθημι, ἀθλοθέτης, ἀνήρ..."
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,{},56,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,...,None,[range],None,None,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,[ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς ...,2,"[[τὶς, φρορὰν, πεντέκοντα, δραχμή, πρᾶχσιν, εν...","[τὶς, φρορὰν, πεντέκοντα, δραχμή, πρᾶχσιν, ενα..."
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,{},6,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,...,None,"[exact, phase, ca]",None,None,ἔδοχσεν τει βολει καὶ τοι δέμοι ὅτε παραιβάτες...,ἔδοχσεν τει βολει καὶ τοι δέμοι ὅτε παραιβάτες...,[ἔδοχσεν τει βολει καὶ τοι δέμοι ὅτε παραιβάτε...,1,"[[ἔδοχσεν, τει, βολει, δέμοι, παραιβάτες, γραμ...","[ἔδοχσεν, τει, βολει, δέμοι, παραιβάτες, γραμμ..."


# Reading the data back

In [4]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_dated_lemmatized_20201214.json", "df", conf)

In [6]:
def count_words(x):
    try:
        return len(x.split())
    except:
        return 0
PHI["clean_text_pythia_ext"].apply(count_words).sum() # previously 4469462, then 4507247

11551090

In [10]:
PHI["clean_text_interpretive_word"].apply(count_words).sum()

4507247

In [9]:
PHI["clean_text_pythia"].apply(count_words).sum() # previously 4469462, then 4507247

8704432

In [45]:
PHI["lemmata"].apply(lambda x: len(x)).sum() # previously 2743932, 2846806, 2867815

7132407